In [3]:
library(pacman)
p_load(tidyverse, data.table, ieugwasr, forestploter, grid)
p_load_gh("MRCIEU/TwoSampleMR")
setwd("/mnt/projects/users/sgok603/pd_mr/scripts/")

## Declare functions

In [4]:
## read tissue GRN
read_tissue_map <- function(dir) {
    #dir <- "../../../../tissue_maps/Brain_Cortex/"
    res <- tibble()
    cortex_map <- file.path(list.files(dir, 
                            pattern = "significant_eqtls.txt", 
                            recursive = TRUE,
                            full.names = TRUE))
    res <- res %>%
            bind_rows(
                read_tsv(cortex_map, show_col_types = FALSE)) %>% 
            dplyr::select(snp, beta, beta_se, alt, ref, maf, eqtl_pval, 
                          gene, interaction_type) %>% 
            distinct()
} 

# create directory if not exists
create_dir <- function(fp){
    if (!dir.exists(fp)){
        dir.create(fp)
    }
    fp
    }

# create output file to write the results
create_file_ifnot_exists <- function(out_fp, grn){
    #grn <- str_replace_all(grn, c("'" = "", " " = "_"))
    
    if (!dir.exists(out_fp)){
        dir.create(out_fp)
    }
    fp <- file.path(out_fp, paste0(grn, "_data_for_exposure.txt"))
    return(fp)
}

## prepare data for running MR
## 1. Prepare exposure data
prepare_exposure_data <- function(eqtl_file, fp){
    if (!dir.exists(fp)){
        dir.create(fp)
    }
    exposure_df <- read_exposure_data(
                        filename = file.path(eqtl_file),
                        sep = "\t",
                        snp_col = "snp",
                        beta_col = "beta",
                        se_col = "beta_se",
                        effect_allele_col = "alt",
                        other_allele_col = "ref",
                        eaf_col = "maf",
                        pval_col = "eqtl_pval",
                        phenotype_col = "gene") %>% 
                   distinct() %>% 
                   write_tsv(file.path(fp, "exposure_df.txt"))
}

## 2. Clump SNPs in the exposure dataset
# run locally 

## 3. Extract outcome data
extract_outcome <- function(exposure_df, gwas, fp){
    outcome_df <- extract_outcome_data(
                        snps = exposure_df$SNP,
                        outcomes = gwas,
                        proxies = TRUE,
                        rsq = 0.9) %>%  
                  distinct() %>% 
                  write_tsv(file.path(fp, "outcome_df.txt"))
}    

## 4. Harmonize exposure outcome effects
harmonise_expo_outcome <- function(exposure_df, outcome_df, fp) {
    harmonised_df <- harmonise_data(
                        exposure_dat = exposure_df, 
                        outcome_dat = outcome_df) %>% 
                     filter(!mr_keep == "FALSE") %>% # Remove SNPs that failed harmonisation
                     distinct() %>%
                     write_tsv(file.path(fp, "harmonised_eo_df.txt")) # using only one gwas study for outcome so power pruning is not required 
}

## 5. Perform MR
## Performs the analysis multiple times for each exposure-outcome combination - each time using a different single SNP to perform the analysis

perform_singlesnp_mr_single <- function(df, method = NULL) {
    if (!is.null(method)) {
        res <- mr_singlesnp(df, single_method = method)
    }
    else {
        res <- mr_singlesnp(df)
    }            
}

## Prepare exposure data

In [ ]:
## analysis are conducted with PD SNPs
## tissue map directory

blood_map_dir <- "/mnt/projects/tissue_maps/Whole_Blood/"
blood_grn_eqtl_info <- read_tissue_map(blood_map_dir) %>% 
                dplyr::select(snp, beta, beta_se, alt, ref, maf, eqtl_pval, gene) %>% 
                filter(eqtl_pval < 1*10^-5) %>% 
                distinct() %>% 
                write_tsv(create_file_ifnot_exists("../data/blood", "blood"))

out_file_path <- '../data/blood'
exposure_data <- prepare_exposure_data(
                    '../data/blood/blood_data_for_exposure.txt',
                    out_file_path
                    )

### clump exposure SNPs

In [6]:
exposure_clumped <- read_tsv("../data/blood/blood_grn_ld_clumped.txt",
                            show_col_types=FALSE) %>% 
print(length(unique(exposure_clumped$SNP))) # 9706
print(length(unique(exposure_clumped$exposure))) # 9091
head(exposure_clumped)

[1] 9705
[1] 9090


SNP,pval.exposure,id.exposure,beta.exposure,se.exposure,effect_allele.exposure,other_allele.exposure,eaf.exposure,exposure,mr_keep.exposure,pval_origin.exposure,data_source.exposure
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<chr>
rs1265960,7.865976e-19,gWrbl7,0.4918905,0.05367653,A,G,0.09850746,A1BG,TRUE,reported,textfile
rs893183,3.224893e-22,O5DX1i,0.6291434,0.06235464,C,T,0.06567162,A1BG-AS1,TRUE,reported,textfile
rs11554674,9.967745e-45,Ms0JNF,0.4910317,0.03215065,A,G,0.20895523,A3GALT2,TRUE,reported,textfile
rs28992176,1.409971e-52,NmlnpV,0.7424393,0.04399376,G,A,0.12462687,A4GALT,TRUE,reported,textfile
rs12600847,8.823841e-26,Ha80Ha,-0.3885843,0.03531311,T,C,0.35373133,AANAT,TRUE,reported,textfile
rs116732850,5.534471e-13,rFuJaA,-0.2061192,0.02795294,G,C,0.06343284,AAR2,TRUE,reported,textfile


### extract outcome data

In [7]:
out_file_path <- '../data/blood/'
outcome_data <- extract_outcome(exposure_clumped, 'ieu-b-7', out_file_path)
length(unique(outcome_data$SNP)) #9394 --> 9106
head(outcome_data)

Extracting data for 9705 SNP(s) from 1 GWAS(s)

Finding proxies for 1444 SNPs in outcome ieu-b-7

Extracting data for 1444 SNP(s) from 1 GWAS(s)

1 of 1 outcomes

 [>] 1 of 3 chunks

 [>] 2 of 3 chunks

 [>] 3 of 3 chunks



[1] 9106

,SNP,chr,pos,beta.outcome,se.outcome,samplesize.outcome,pval.outcome,eaf.outcome,effect_allele.outcome,other_allele.outcome,⋯,outcome.deprecated,mr_keep.outcome,data_source.outcome,proxy.outcome,target_snp.outcome,proxy_snp.outcome,target_a1.outcome,target_a2.outcome,proxy_a1.outcome,proxy_a2.outcome
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,⋯,<chr>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,rs10937106,3,182738287,0.1109,0.0190,482730,0.000000005,0.7153,A,G,⋯,Parkinson's disease || ||,TRUE,igd,NA,NA,NA,NA,NA,NA,NA
2,rs11711420,3,183349010,0.0260,0.0194,482730,0.181400000,0.2660,G,T,⋯,Parkinson's disease || ||,TRUE,igd,NA,NA,NA,NA,NA,NA,NA
3,rs8478,3,183899832,-0.0106,0.0215,482730,0.622801000,0.1904,T,C,⋯,Parkinson's disease || ||,TRUE,igd,NA,NA,NA,NA,NA,NA,NA
4,rs843336,3,183912339,-0.0124,0.0175,482730,0.478900000,0.5216,G,A,⋯,Parkinson's disease || ||,TRUE,igd,NA,NA,NA,NA,NA,NA,NA
5,rs1981767,3,186649103,0.0129,0.0242,471013,0.594700000,0.2879,A,G,⋯,Parkinson's disease || ||,TRUE,igd,NA,NA,NA,NA,NA,NA,NA
6,rs13315299,3,192805621,0.0378,0.0270,466555,0.162100000,0.2669,T,C,⋯,Parkinson's disease || ||,TRUE,igd,NA,NA,NA,NA,NA,NA,NA


In [10]:
colnames(outcome_data)

[1] "SNP"                   "chr"                   "pos"                  
 [4] "beta.outcome"          "se.outcome"            "samplesize.outcome"   
 [7] "pval.outcome"          "eaf.outcome"           "effect_allele.outcome"
[10] "other_allele.outcome"  "outcome"               "id.outcome"           
[13] "originalname.outcome"  "outcome.deprecated"    "mr_keep.outcome"      
[16] "data_source.outcome"   "proxy.outcome"         "target_snp.outcome"   
[19] "proxy_snp.outcome"     "target_a1.outcome"     "target_a2.outcome"    
[22] "proxy_a1.outcome"      "proxy_a2.outcome"

### harmonise exposure/outcome data

In [10]:
out_file_path <- '../data/blood/'
# harmonised_df <- harmonise_expo_outcome(exposure_clumped, outcome_data, out_file_path)
length(unique(harmonised_df$SNP)) # 8791
length(unique(harmonised_df$exposure)) # 8305

[1] 8791

[1] 8305

### mr input data

In [13]:
mr_input_data <- read_tsv("../data/blood/harmonised_eo_df.txt",
                         show_col_types = FALSE) 
length(unique(mr_input_data$exposure))
length(unique(mr_input_data$SNP))

[1] 8305

[1] 8791

#### separate exposure with single and multiple instruments

In [14]:
# exposure with single SNP instrument
single_snp_exposures <- mr_input_data %>% 
                group_by(exposure) %>% 
                summarise(instruments = length(SNP)) %>% 
                filter(instruments == 1)

nrow(single_snp_exposures) #7543
length(unique(single_snp_exposures$exposure)) #7543

# exposure with multiple SNP instruments
multiple_snp_exposures <- mr_input_data %>% 
                group_by(exposure) %>% 
                summarise(instruments = length(SNP)) %>% 
                filter(instruments > 1)
nrow(multiple_snp_exposures) #762
length(unique(multiple_snp_exposures$exposure)) #762


[1] 7543

[1] 7543

[1] 762

[1] 762

#### perform sensitivity analysis for exposures with multiple snp instruments

In [15]:
## do sensitivity analysis (remove Q_pval < 0.05)
hetero_test_passed <- mr_heterogeneity(mr_input_data %>% 
                                filter(exposure %in% multiple_snp_exposures$exposure) %>% 
                                distinct()) %>% 
                        filter(!Q_pval < 0.05)
length(unique(hetero_test_passed$exposure)) #713
# hetero_test_passed

## do horizontal pleiotropy test (remove p-val > 0.05 or NA)
pleiotropy_test_passed <- mr_pleiotropy_test(mr_input_data %>% 
                                filter(exposure %in% multiple_snp_exposures$exposure) %>% 
                                distinct()) %>% 
                         filter(!pval == "NA" | pval > 0.05) 
length(unique(pleiotropy_test_passed$exposure)) #58
# pleiotropy_test_passed

[1] 713

[1] 58

In [16]:
# exposures that have no horizontal pleotropic effect and heterogeneity from the above tests
exposures_for_ivw <- hetero_test_passed %>% 
                    distinct(exposure) %>% 
                    inner_join(pleiotropy_test_passed %>% 
                              distinct(exposure), by = "exposure")

### perform wald ratio MR analysis

In [17]:
## perform MR for exposure with single instrument
mr_input_data_wald <- mr_input_data %>% 
                        filter(exposure %in% single_snp_exposures$exposure) %>% 
                        distinct()
 
wald_mr_res <- perform_singlesnp_mr_single(mr_input_data_wald, "mr_wald_ratio") #%>% 
                # write_tsv("../data/blood/blood_grn_full_mr_res_wald.txt")

### generate odds ratios for MR results

In [18]:
wald_mr_res <- read_tsv("../data/blood/blood_grn_full_mr_res_wald.txt",
                       show_col_types = FALSE)


## Perform MR strong effects
wald_mr_res_strong <- wald_mr_res %>% 
                        filter(!is.na(p)) %>% 
                        generate_odds_ratios() %>% 
                        filter(p < 0.05/n_distinct(exposure)) #%>% 
                        # write_tsv("../results/blood/blood_grn_wald_strong.txt")

## Perform MR suggestive effects
wald_mr_res_sugg <- wald_mr_res %>% 
                        filter(!is.na(p)) %>%
                        generate_odds_ratios() %>%
                        filter(p > 0.05/n_distinct(exposure) & p < 0.05) #%>% 
                        # write_tsv("../results/blood/blood_grn_wald_sugg.txt")

### perform ivw (inverse variance weighted) MR analysis

In [19]:
## perform MR for exposure with multiple instruments
mr_input_data_ivw <- mr_input_data %>% 
                        filter(exposure %in% exposures_for_ivw$exposure) %>% 
                        distinct() 

# Perform MR strong effects IVW
mr_res_ivr_strong <- perform_singlesnp_mr_single(mr_input_data_ivw, "mr_ivw") %>% 
                        filter(!is.na(p) & !SNP == "All - MR Egger") %>% 
                        generate_odds_ratios() %>% 
                        filter(p < 0.05/n_distinct(exposure)) %>% 
                        distinct() 

mr_res_ivr_strong_with_snps <- mr_res_ivr_strong %>% 
    select(-SNP) %>% 
    inner_join(mr_input_data_ivw %>% 
              select(SNP, exposure) %>% 
              distinct(), by = "exposure") %>% 
    group_by(exposure) %>% 
    mutate(snp = paste0(unique(SNP), collapse = ",")) %>% 
    select(-SNP) %>% 
    distinct()# %>% 
    # write_tsv("../results/blood/blood_grn_ivw_strong.txt")

# ## Perform MR suggestive effects IVW
mr_res_ivr_sugg <- perform_singlesnp_mr_single(mr_input_data_ivw, "mr_ivw") %>% 
                        filter(!is.na(p) & !SNP == "All - MR Egger") %>% 
                        generate_odds_ratios() %>% 
                        filter(p > 0.05/n_distinct(exposure) & p < 0.05) %>% 
                        distinct()

mr_res_ivr_sugg_with_snps <- mr_res_ivr_sugg %>% 
    select(-SNP) %>% 
    inner_join(mr_input_data_ivw %>% 
              select(SNP, exposure) %>% 
              distinct(), by = "exposure") %>% 
    group_by(exposure) %>% 
    mutate(snp = paste0(unique(SNP), collapse = ",")) %>% 
    select(-SNP) %>% 
    distinct() #%>% 
    # write_tsv("../results/blood/blood_grn_ivw_sugg.txt")